**FEEL FREE TO UPVOTE**  （＾ｖ＾）

In [ ]:

#v20提交成绩得到了3.688， cv 成绩3.6519
#使用authorized_flag 进行特征分别处理,收到了奇效
#使用countVectorizer 对类别变量进行编码，内存存在压力

In [163]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
import time
warnings.filterwarnings('ignore')
np.random.seed(4950)

In [164]:
import pickle

In [165]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [166]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_hist_trans = pd.read_csv('../../ELO/historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('../../ELO/new_merchant_transactions.csv')

In [167]:
df_hist_trans = reduce_mem_usage(df_hist_trans)
df_new_merchant_trans = reduce_mem_usage(df_new_merchant_trans)

Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [168]:
df_hist_trans.purchase_date=pd.to_datetime(df_hist_trans.purchase_date)
pur_date=df_hist_trans[df_hist_trans.month_lag==0].groupby('card_id').purchase_date.max()
card_id_nan_unique=df_hist_trans[df_hist_trans.card_id.isin(pur_date.index)==False].card_id.unique()
df=pd.DataFrame(card_id_nan_unique)
df['month_lag_date']=pd.to_datetime('2018-02') # Seetting all nan values to 2018 Feb
df.set_index(0,inplace=True)
new_map=df.month_lag_date
method=pur_date.append(new_map)
del df,pur_date
gc.collect()
time.sleep(5)

In [169]:
for df in [df_hist_trans,df_new_merchant_trans]:
#     df['category_2'].fillna(1.0,inplace=True)
#     df['category_3'].fillna('A',inplace=True)
#     df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    #修改特征
    df['category_2'].fillna(-1,inplace=True)
    df['category_3'].fillna('other',inplace=True)
    df['merchant_id'].fillna('other',inplace=True)
#     df.loc[df['installments'].isin([999,-1]),'installments'] = 0 

In [170]:
def get_new_columns(name,aggs):
    #for for 写法 nice
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [171]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    
    
    ##Think about this part!
    ## This line of code may be missleading since there is no single ref date. 
    df['month_diff'] = ((datetime.datetime(2018,6,1) - df['purchase_date']).dt.days)//30 # original code
    df['month_diff'] += df['month_lag']
    ## removing 'month_diff' reduces LB score to 3.682
    ## This line of code can be also missleading since for new merchant hist it might be negative. But why we care about it.
    #df['month_diff'] = ((df['card_id'].map(method) - df['purchase_date']).dt.days)//30 # method that I used in my code
    
    ##But we already have mongth lag for each purchase! So we don't really need it!
    
    
    # This makes it even more irrelevant for my case since it will be equal to zero in most of the cases.So, let's remove it for my case
    #df['month_diff'] += df['month_lag']

In [172]:
(pd.to_datetime('2018-12-30')-pd.to_datetime('2019-12-30')).days//30

-13

In [173]:
df_new_merchant_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,category_2,state_id,subsector_id,year,weekofyear,month,dayofweek,weekend,hour,month_diff
0,1,C_ID_415bb3a509,107,0,1,B,307,M_ID_b0c793002c,1,-0.557617,...,1.0,9,19,2018,10,3,6,1,14,3
1,1,C_ID_415bb3a509,140,0,1,B,307,M_ID_88920c89e8,1,-0.569336,...,1.0,9,19,2018,12,3,0,0,18,3
2,1,C_ID_415bb3a509,330,0,1,B,507,M_ID_ad5237ef6b,2,-0.551270,...,1.0,9,14,2018,17,4,3,0,14,3
3,1,C_ID_415bb3a509,-1,1,1,B,661,M_ID_9e84cda3b1,1,-0.671875,...,-1.0,-1,8,2018,10,3,2,0,9,3
4,1,C_ID_ef55cf8d4b,-1,1,1,B,166,M_ID_3c86fa3831,1,-0.659668,...,-1.0,-1,29,2018,12,3,3,0,21,3


In [174]:
#因为根据auth_flag 将特征分成了两部分，这里聚合一个全局的auth_flag 特征
aggs = {}
aggs['purchase_amount'] = ['sum']
# aggs['installments'] = ['sum','max','min','mean','var','median']
aggs['authorized_flag'] = ['sum', 'mean','std']
aggs['card_id'] = ['size']
auth_flag = df_hist_trans.groupby(['card_id']).agg(aggs)
auth_flag.columns = get_new_columns('auth_flag',aggs)
auth_flag.reset_index(inplace=True)
df_train = df_train.merge(auth_flag,on='card_id',how='left')
df_test = df_test.merge(auth_flag,on='card_id',how='left')
del auth_flag
gc.collect()

91

In [175]:
#对 authorized_flag进行结果编码
aggs = {}
for col in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
    df_hist_trans[col+'_auth_mean'] = df_hist_trans.groupby([col])['authorized_flag'].transform('mean')
    df_hist_trans[col+'_auth_sum'] = df_hist_trans.groupby([col])['authorized_flag'].transform('sum') 
    aggs[col+'_auth_mean'] = ['mean']
    aggs[col+'_auth_sum'] = ['sum'] 
auth_encoder = df_hist_trans.groupby(['card_id']).agg(aggs)
auth_encoder.columns = get_new_columns('auth_encoder',aggs)
auth_encoder.reset_index(inplace=True)
df_train = df_train.merge(auth_encoder,on='card_id',how='left')
df_test = df_test.merge(auth_encoder,on='card_id',how='left')
del auth_encoder
gc.collect()

35

In [176]:
def aggregate_per_month(prefix,history,agg_func):
    grouped = history.groupby(['card_id', 'month_lag'])
    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = [prefix + '_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = [prefix + '_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True) 
    return final_group

In [177]:
#对授权码 进行按月聚合
agg_func = {'authorized_flag': [ 'sum', 'mean','median']}
final_group =  aggregate_per_month('agg_per_month_total',df_hist_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group

In [178]:
authorized_transactions = df_hist_trans[df_hist_trans['authorized_flag'] == 1]
df_hist_trans = df_hist_trans[df_hist_trans['authorized_flag'] == 0]

In [179]:
#使用countVectorizer对category特征进行处理，别人号称可以提升 千分之3
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(token_pattern='\w{1,}')   

In [180]:
df_hist_trans.shape

(2516909, 33)

In [181]:
agg_func = {'purchase_amount': [ 'sum', 'mean', 'min', 'max', 'std']}
final_group =  aggregate_per_month('agg_per_month_auth',authorized_transactions,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
final_group =  aggregate_per_month('agg_per_month_hist',df_hist_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
final_group =  aggregate_per_month('agg_per_month_hist_new',df_new_merchant_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
gc.collect()

99

In [182]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [183]:
i = 0
for df in [authorized_transactions,df_hist_trans]:
    aggs = {}
    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
                'state_id','city_id']:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var','median']
    aggs['installments'] = ['sum','max','min','mean','var','median']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var','median']
    
    ## I also remove this part since it is also irrelevant since it is equivelent to month_lag. But it makes it worser
    aggs['month_diff'] = ['mean','median']



#     aggs['authorized_flag'] = ['sum', 'mean','median']
    aggs['weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']
    #产生交叉特征，内存有问题
    features = ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id'
               ,'merchant_id']
#     for coli in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#         for colj in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#             df[coli + colj] = df[coli].astype(str) + df[colj].astype(str)
#             features.append(coli + colj)
    for col in features:
        df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum') 
        aggs[col+'_mean'] = ['mean']
        aggs[col+'_sum'] = ['sum'] 
#         添加特征，使用outlier进行编码
#         outliers_mean = df.groupby([col])['outliers'].mean()
#         outliers_sum = df.groupby([col])['outliers'].sum()
#         df[col+'_outliers_mean'] = df[col].map(outliers_mean)
#         df[col+'_outliers_sum'] = df[col].map(outliers_sum)
#         aggs[col+'_outliers_mean'] = ['mean']
#         aggs[col+'_outliers_sum'] =['sum']   
        
    if i == 0:
        prefix = 'auth_hist'
    else:
        prefix = 'hist'
    new_columns = get_new_columns(prefix,aggs)
    i += 1
    # df_hist_trans.sort_values(['card_id','purchase_date'],inplace = True)

    df_hist_trans_group = df.groupby('card_id').agg(aggs)
    df_hist_trans_group.columns = new_columns
    df_hist_trans_group.reset_index(drop=False,inplace=True)
    df_hist_trans_group[prefix + '_purchase_date_diff'] = (df_hist_trans_group[prefix + '_purchase_date_max'] - df_hist_trans_group[prefix + '_purchase_date_min']).dt.days
    df_hist_trans_group[prefix + '_purchase_date_average'] = df_hist_trans_group[prefix + '_purchase_date_diff']/df_hist_trans_group[prefix + '_card_id_size']
    
    
    ###Think about below line of code  
    
    
    ###up to now for history means month_lag_0 to purdate_min
#     df_hist_trans_group[prefix + '_purchase_date_uptonow'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group[prefix + '_purchase_date_max']).dt.days
    df_hist_trans_group[prefix + '_purchase_date_uptonow'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group[prefix + '_purchase_date_min']).dt.days
    df_hist_trans_group[prefix + '_first_purchase'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group[prefix + '_purchase_date_max']).dt.days
  
    
    
    
    #下面这个特征考虑了：有的人可能购买频率较低，但是还是忠实粉丝的情况
#     df_hist_trans_group[prefix + '_purchase_date_uptonow_ave'] =  df_hist_trans_group[prefix + '_purchase_date_uptonow']/df_hist_trans_group[prefix + '_purchase_date_average']

    #每一个card中未授权消费次数
#     df_hist_trans_group[prefix + '_unauthorized_number'] = df_hist_trans_group[prefix + '_card_id_size'] - df_hist_trans_group[prefix + '_authorized_flag_sum']
    #最近活跃时间，确实是一个强特征，感觉可以再挖出来几个特征，比如最近5次消费时间，最近10次消费时间，如果值比较小，说明最近很活跃
    #没有效果
    # grouped =  df_hist_trans.groupby('card_id')['purchase_date']
    # df_hist_trans_group['hist_purchase_5thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(5)).dt.days
    # df_hist_trans_group['hist_purchase_3thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(3)).dt.days
    # df_hist_trans_group['hist_purchase_10thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(10)).dt.days
    df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
    df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
    del df_hist_trans_group
    time.sleep(5)
    gc.collect()
    del df;gc.collect()
    time.sleep(5)

In [184]:
aggs = {}
#添加特征
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
            'state_id','city_id']:
    aggs[col] = ['nunique']
    
aggs['purchase_amount'] = ['sum','max','min','mean','var','median']
aggs['installments'] = ['sum','max','min','mean','var','median']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var','median']

### Now we also need to remove from here since we already have month_lag
aggs['month_diff'] = ['mean','median']


aggs['weekend'] = ['sum', 'mean']
aggs['authorized_flag'] = ['sum', 'mean','median']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

#添加特征   
features = ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id'
               ,'merchant_id']
#产生交叉特征，内存有问题
# for coli in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#     for colj in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#         df_new_merchant_trans[coli + colj] = df_new_merchant_trans[coli].astype(str) + df_new_merchant_trans[colj].astype(str)
#         features.append(coli + colj)
for col in features:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    df_new_merchant_trans[col+'_sum'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('sum')
    aggs[col+'_mean'] = ['mean']
    aggs[col+'_sum'] = ['sum']
    #添加特征，使用outlier进行编码
#     outliers_mean = df.groupby([col])['outliers'].mean()
#     outliers_sum = df.groupby([col])['outliers'].sum()
#     df[col+'_outliers_mean'] = df[col].map(outliers_mean)
#     df[col+'_outliers_sum'] = df[col].map(outliers_sum)
#     aggs[col+'_outliers_mean'] = ['mean']
#     aggs[col+'_outliers_sum'] =['sum']   
    
new_columns = get_new_columns('new_hist',aggs)
# df_new_merchant_trans.sort_values(['card_id','purchase_date'],inplace = True)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
# it is fine for new mech transaction history
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group['new_hist_purchase_date_max']).dt.days

# new feature for the first purchase
df_hist_trans_group['new_hist_first_purchase'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group['new_hist_purchase_date_min']).dt.days

#下面这个特征考虑了：有的人可能购买频率较低，但是还是忠实粉丝的情况
# df_hist_trans_group['new_hist_purchase_date_uptonow_ave'] =  df_hist_trans_group['new_hist_purchase_date_uptonow']/df_hist_trans_group['new_hist_purchase_date_average']

#每一个card中未授权消费次数
df_hist_trans_group['new_hist_unauthorized_number'] = df_hist_trans_group['new_hist_card_id_size'] - df_hist_trans_group['new_hist_authorized_flag_sum']
# grouped = df_new_merchant_trans.groupby('card_id')['purchase_date']
# df_hist_trans_group['new_hist_purchase_5thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(5)).dt.days
# df_hist_trans_group['new_hist_purchase_3thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(3)).dt.days
# df_hist_trans_group['new_hist_purchase_10thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(10)).dt.days

df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group
gc.collect()
del df_new_merchant_trans
gc.collect()
time.sleep(5)

In [185]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (df['card_id'].map(method) - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    #添加特征
    df['auth_hist_first_buy'] = (df['auth_hist_purchase_date_min'] - df['first_active_month']).dt.days
    #修改特征
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min','auth_hist_purchase_date_max','auth_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    for f in ['auth_encoder_category_2_auth_sum_sum','auth_encoder_category_3_auth_sum_sum',
            'auth_encoder_state_id_auth_sum_sum','auth_encoder_subsector_id_auth_sum_sum',
            'auth_encoder_merchant_category_id_auth_sum_sum','auth_encoder_city_id_auth_sum_sum']:
        df[f] = df[f].astype(np.int64) * 1e-9
    #上面auth_flag已经聚合过了card_id_size ,purchase_amount
#     df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']  + df['auth_hist_card_id_size']
#     df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']+df['auth_hist_purchase_amount_sum']
#添加特征

for f in ['feature_1','feature_2','feature_3','month','dayofweek']:
    order_label1 = df_train.groupby([f])['outliers'].mean()
    df_train[f+'_outliers_mean'] = df_train[f].map(order_label1)
    df_test[f+'_outliers_mean'] = df_test[f].map(order_label1)
    
    order_label2 = df_train.groupby([f])['outliers'].sum()
    df_train[f+'_outliers_sum'] = df_train[f].map(order_label2)
    df_test[f+'_outliers_sum'] = df_test[f].map(order_label2)
    
#     order_label1 = df_train.groupby([f])['target'].mean()
#     df_train[f+'_target_mean'] = df_train[f].map(order_label1)
#     df_test[f+'_target_sum'] = df_test[f].map(order_label1)
#     order_label2 = df_train.gorupby([f])['target'].sum()
#     df_train[f+'_target_sum'] = df_train[f].map(order_label2)
#     df_test[f+'_target_sum'] = df_test[f].map(order_label2)
 
# get_dummies 似乎有一点点不良影响
df_train = pd.get_dummies(df_train,columns =['feature_1','feature_2'])
df_test = pd.get_dummies(df_test,columns =['feature_1','feature_2'])

In [186]:
#首次购买的时间居然早于首次激活的时间，进行调整
df_train.loc[df_train['auth_hist_first_buy'] < 0, 'auth_hist_first_buy'] = -1
df_train.loc[df_train['hist_first_buy'] < 0, 'hist_first_buy'] = -1
df_train.head()

,first_active_month,card_id,feature_3,target,auth_flag_purchase_amount_sum,auth_flag_authorized_flag_sum,auth_flag_authorized_flag_mean,auth_flag_authorized_flag_std,auth_flag_card_id_size,auth_encoder_category_2_auth_mean_mean,...,dayofweek_outliers_mean,dayofweek_outliers_sum,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3
0,2017-06-01,C_ID_92a2005557,1,-0.820283,-165.968735,247,0.950000,0.218365,260,0.927583,...,0.015174,401,0,0,0,0,1,0,1,0
1,2017-01-01,C_ID_3d0044924f,0,0.392913,-210.006332,339,0.968571,0.174723,350,0.914943,...,0.009631,289,0,0,0,1,0,1,0,0
2,2016-08-01,C_ID_d639edf6cd,0,0.688056,-29.167391,41,0.953488,0.213083,43,0.933632,...,0.013597,247,0,1,0,0,0,0,1,0
3,2017-09-01,C_ID_186d6a6901,0,0.142495,-49.491364,77,1.000000,0.000000,77,0.905491,...,0.005835,201,0,0,0,1,0,0,0,1
4,2017-11-01,C_ID_cdbd2c0db2,0,-0.159749,-48.687656,128,0.962406,0.190931,133,0.911664,...,0.011268,409,1,0,0,0,0,0,0,1


In [187]:
exclude_features = []
exclude_features += ['card_id', 'first_active_month','target','outliers']
df_train_columns = [c for c in df_train.columns if c not in exclude_features ]
target = df_train['target']
del df_train['target']

In [188]:
len(df_train_columns)
len(exclude_features)

4

In [189]:
data_type=df_train.dtypes

In [190]:
data_type[(data_type!='float32')&(data_type!='float64')&(data_type!='float16')]

first_active_month                        datetime64[ns]
card_id                                           object
feature_3                                          int64
auth_flag_authorized_flag_sum                      int64
auth_flag_card_id_size                             int64
outliers                                           int64
auth_hist_month_nunique                            int64
auth_hist_hour_nunique                             int64
auth_hist_weekofyear_nunique                       int64
auth_hist_dayofweek_nunique                        int64
auth_hist_year_nunique                             int64
auth_hist_subsector_id_nunique                     int64
auth_hist_merchant_id_nunique                      int64
auth_hist_merchant_category_id_nunique             int64
auth_hist_state_id_nunique                         int64
auth_hist_city_id_nunique                          int64
auth_hist_installments_sum                         int16
auth_hist_installments_max     

## Above is the same with wang, I call it wang's features

### I replace wang's stratified CV with normal lgb.cv

In [191]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 48,
         "random_state": 4950}

In [192]:
trn_data = lgb.Dataset(df_train[df_train_columns], label=target)#, categorical_feature=categorical_feats)

In [ ]:
#trn_data = lgb.Dataset(df_train[df_train_columns], label=target, categorical_feature=['month','dayofweek','weekofyear'])

In [193]:
lgb_cv = lgb.cv(param, trn_data, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

[200]	cv_agg's rmse: 3.68297 + 0.0296405
[400]	cv_agg's rmse: 3.66285 + 0.0279173
[600]	cv_agg's rmse: 3.6557 + 0.0269269
[800]	cv_agg's rmse: 3.65282 + 0.0268649
[1000]	cv_agg's rmse: 3.65211 + 0.0266568
[1200]	cv_agg's rmse: 3.65155 + 0.025996
[1400]	cv_agg's rmse: 3.65171 + 0.0256495
[1600]	cv_agg's rmse: 3.65192 + 0.0254616
[1800]	cv_agg's rmse: 3.65214 + 0.0251097


In [ ]:
lgb_cv

Results without categorical_feature=['month','dayofweek','weekofyear'].
[200]	cv_agg's rmse: 3.69075 + 0.0319068
[400]	cv_agg's rmse: 3.6695 + 0.0304884
[600]	cv_agg's rmse: 3.66173 + 0.0294372
[800]	cv_agg's rmse: 3.65849 + 0.0290465
[1000]	cv_agg's rmse: 3.65738 + 0.0285758
[1200]	cv_agg's rmse: 3.65658 + 0.0280419
[1400]	cv_agg's rmse: 3.65644 + 0.0274495
[1600]	cv_agg's rmse: 3.65649 + 0.0271829
[1800]	cv_agg's rmse: 3.65663 + 0.0268914

In [194]:
lgb_cv['rmse-mean'].index(min(lgb_cv['rmse-mean']))

1246

In [195]:
min(lgb_cv['rmse-mean'])

3.651478372059267

In [135]:
len(lgb_cv['rmse-mean'])

1354

In [136]:
lgb_cv.keys()

dict_keys(['rmse-mean', 'rmse-stdv'])

Original wang results:

[200]	cv_agg's rmse: 3.68161 + 0.0299797
[400]	cv_agg's rmse: 3.66225 + 0.0286057
[600]	cv_agg's rmse: 3.65503 + 0.0277111
[800]	cv_agg's rmse: 3.65259 + 0.0276703
[1000]	cv_agg's rmse: 3.65156 + 0.027532
[1200]	cv_agg's rmse: 3.65109 + 0.0273339
[1400]	cv_agg's rmse: 3.65112 + 0.0268877
[1600]	cv_agg's rmse: 3.65121 + 0.0265736
[1800]	cv_agg's rmse: 3.65127 + 0.0266131

lgb.cv show's that num_boost_round is around 2000, so we train a model with num_boost_round=2000

In [200]:
clf = lgb.train(param, trn_data, 1400, valid_sets=(trn_data), verbose_eval=200)

[200]	training's rmse: 3.57298
[400]	training's rmse: 3.4858
[600]	training's rmse: 3.42977
[800]	training's rmse: 3.38639
[1000]	training's rmse: 3.3491
[1200]	training's rmse: 3.31491
[1400]	training's rmse: 3.28285


Results without categorical_feature=['month','dayofweek','weekofyear'].

[200]	training's rmse: 3.58917
[400]	training's rmse: 3.49676
[600]	training's rmse: 3.43963
[800]	training's rmse: 3.39575
[1000]	training's rmse: 3.35879
[1200]	training's rmse: 3.32572
[1400]	training's rmse: 3.29513
[1600]	training's rmse: 3.26594
[1800]	training's rmse: 3.23792
[2000]	training's rmse: 3.21083

Original Wang's results:
[200]	training's rmse: 3.57275
[400]	training's rmse: 3.48472
[600]	training's rmse: 3.42865
[800]	training's rmse: 3.3851
[1000]	training's rmse: 3.34906
[1200]	training's rmse: 3.3155
[1400]	training's rmse: 3.28252
[1600]	training's rmse: 3.25342
[1800]	training's rmse: 3.22531
[2000]	training's rmse: 3.19685

predict and save

In [201]:
y_pred = clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration)

pd.DataFrame({
        'card_id': df_test.card_id,
        'target': y_pred
}).to_csv('wang_lgb_cv_pur_date_1_updated2', index=False)

## Outlier detection using wang's features

the param is same as regression and just adjust to binary classification

best num_boost_round is 500, so we use 500 to train a classifer

In [205]:
outlier = np.zeros(len(target), dtype=np.int)

In [206]:
outlier[target < -33] = 1

In [207]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 48,
         "scale_pos_weight": 15,
         "random_state": 4950}

In [208]:
trn_data = lgb.Dataset(df_train[df_train_columns], label=outlier)#, categorical_feature=categorical_feats)

In [209]:
lgb_cv = lgb.cv(param, trn_data, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

[200]	cv_agg's auc: 0.904922 + 0.00481107
[400]	cv_agg's auc: 0.905247 + 0.00477944
[600]	cv_agg's auc: 0.904767 + 0.00481693
[800]	cv_agg's auc: 0.904029 + 0.0050695


In [214]:
lgb_cv['auc-mean'][-1]

0.9054550547804272

In [210]:
clf = lgb.train(param, trn_data, 500, verbose_eval=100, valid_sets=(trn_data))

[100]	training's auc: 0.95459
[200]	training's auc: 0.962208
[300]	training's auc: 0.968107
[400]	training's auc: 0.972841
[500]	training's auc: 0.976854


In [215]:
te_prob = clf.predict(df_test[df_train_columns])

In [216]:
te_prob.min(), te_prob.max()

(0.000454082447820393, 0.8599349095710359)

In [219]:
wang_out=pd.read_csv('wang_lgb_cv_pur_date_1_updated')
wang_out['outlier_prob1'] = te_prob

In [221]:
_40=wang_out.sort_values('outlier_prob1',ascending=False)

In [222]:
top20 = _40.card_id.values[:20].tolist()

In [223]:
wang_out.loc[wang_out.card_id.isin(top20), 'target'] = -33.21928095
wang_out[['card_id', 'target']].to_csv('wang_lgb_cv_pur_date_1_updated_top20.csv', index=False)

In [224]:
top20_50 = _40.card_id.values[20:50].tolist()

In [225]:
top20_50_df = wang_out[wang_out.card_id.isin(top20_50)]

In [226]:
top20_50_target = top20_50_df.outlier_prob1 * (-33.21928095)

In [227]:
wang_out.loc[wang_out.card_id.isin(top20_50), 'target'] = top20_50_target

In [230]:
wang_out[['card_id', 'target']].to_csv('wang_lgb_cv_pur_date_1_updated_top20_top20-50.csv', index=False)

### Now let's try to merge results with the model without outliers.
### First, predict without outliers

In [243]:
df_train['target']=target

In [246]:
df_train2=df_train[df_train['outliers']==0]

In [247]:
df_train2.shape

(199710, 257)

In [248]:
target2=df_train2.target

In [237]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 48,
         "random_state": 4950}

In [249]:
trn_data2 = lgb.Dataset(df_train2[df_train_columns], label=target2)#, categorical_feature=categorical_feats)

In [250]:
lgb_cv = lgb.cv(param, trn_data2, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

[200]	cv_agg's rmse: 1.58191 + 0.00883915
[400]	cv_agg's rmse: 1.56523 + 0.00798252
[600]	cv_agg's rmse: 1.55947 + 0.0075402
[800]	cv_agg's rmse: 1.5571 + 0.00732745
[1000]	cv_agg's rmse: 1.55593 + 0.0070912
[1200]	cv_agg's rmse: 1.55515 + 0.00691628
[1400]	cv_agg's rmse: 1.55474 + 0.00682905
[1600]	cv_agg's rmse: 1.55452 + 0.00677001
[1800]	cv_agg's rmse: 1.55431 + 0.00669755
[2000]	cv_agg's rmse: 1.55412 + 0.00663288
[2200]	cv_agg's rmse: 1.55398 + 0.00654708
[2400]	cv_agg's rmse: 1.55386 + 0.0064701
[2600]	cv_agg's rmse: 1.55382 + 0.00645527
[2800]	cv_agg's rmse: 1.55381 + 0.00636882
[3000]	cv_agg's rmse: 1.55379 + 0.00623851
[3200]	cv_agg's rmse: 1.55373 + 0.00625909
[3400]	cv_agg's rmse: 1.55379 + 0.00623813
[3600]	cv_agg's rmse: 1.55377 + 0.00623746


In [253]:
len(lgb_cv['rmse-mean'])

3181

In [255]:
clf = lgb.train(param, trn_data2, 3600, valid_sets=(trn_data2), verbose_eval=200)

[200]	training's rmse: 1.57406
[400]	training's rmse: 1.54932
[600]	training's rmse: 1.53523
[800]	training's rmse: 1.52464
[1000]	training's rmse: 1.51585
[1200]	training's rmse: 1.50798
[1400]	training's rmse: 1.50061
[1600]	training's rmse: 1.49351
[1800]	training's rmse: 1.48676
[2000]	training's rmse: 1.48012
[2200]	training's rmse: 1.47368
[2400]	training's rmse: 1.46731
[2600]	training's rmse: 1.461
[2800]	training's rmse: 1.45495
[3000]	training's rmse: 1.44899
[3200]	training's rmse: 1.4431
[3400]	training's rmse: 1.43728
[3600]	training's rmse: 1.43142


In [256]:
y_pred = clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration)

pd.DataFrame({
        'card_id': df_test.card_id,
        'target': y_pred
}).to_csv('wang_lgb_cv_pur_date_1_updated_without_outliers', index=False)

### Let's combine models with and without outliers

In [257]:
model_with_outliers=pd.read_csv('wang_lgb_cv_pur_date_1_updated')
model_without_outliers=pd.read_csv('wang_lgb_cv_pur_date_1_updated_without_outliers')


In [266]:
probability_df=model_with_outliers.copy()
probability_df['prob']=te_prob
del probability_df['target']

In [273]:
probability_df.sort_values('prob',ascending=False).head(18000).prob.min()

0.14345419555889208

In [274]:
probability_df.sort_values('prob',ascending=False).head(18000).prob.max()

0.8599349095710359

In [282]:
#Let's use 18000 as threshould value as we have used before
index_high_prob=probability_df.sort_values('prob',ascending=False).head(18000).index

In [276]:
index_high_prob

Int64Index([ 96354,  88754,  78078, 115682, 109960,   5008, 109705, 118268,
              5707,  57626,
            ...
            116990,  53485,  88674,  88770,  52048, 115876,  51772,  26502,
             11497,  66954],
           dtype='int64', length=18000)

In [292]:
mixed_model=model_without_outliers.copy()
mixed_model.loc[index_high_prob,'target']=model_with_outliers.loc[index_high_prob,'target']
mixed_model.to_csv('wang_lgb_cv_pur_date_1_updated_mixed.csv',index=False)

### Now let's use previous methods to change top 20 and top 20-50 results

In [297]:
mixed_model.loc[mixed_model.card_id.isin(top20),'target']=-33.21928095

In [301]:
mixed_model.loc[mixed_model.card_id.isin(top20_50),'target']=top20_50_target

In [305]:
mixed_model.to_csv('wang_lgb_cv_pur_date_1_updated_mixed_top20_top20_50.csv',index=False)

##LB score: 3.663